In [1]:
%load_ext Cython

In [2]:
%%cython
import itertools
import time
import copy
from memory_profiler import memory_usage
from collections import Counter

In [3]:
support = 2
F = []
input_file = 'contextPasquier99.txt'
output_file = open('outfile_prepostplus_simple', 'w')
all_n_list = {}

In [4]:
def make_itemset_1(filename):
    global support
    itemset_1 = {}
    with open(filename) as finput:
        f = finput.read().split('\n')
        for line in f:
            items = line.strip().split(' ')
            for item in items:
                if item not in itemset_1:
                    itemset_1[item] = 0
                itemset_1[item] += 1
    
    for k, v in list(itemset_1.items()):
        if v < support:
            del itemset_1[k]
    return itemset_1

In [5]:
class PPCNode:
    def __init__(self):
        self.pre_order = None
        self.post_order = None
        self.count = None
        self.label = None
        self.parent = None
        self.child = None
        self.sibling = None


class FPTNode:
    def __init__(self):
        self.equivalent_items = None
        self.child_nodes = None
        self.label = None
        self.itemset = None
        self.support = None

In [6]:
def create_node(root, label):
    node = PPCNode()
    node.label = label
    node.count = 1
    node.parent = root
    return node


def insert_ppc_node(root, item):
    if root.child is None:
        node = create_node(root, item)
        root.child = node
        return root.child
    elif root.child.label == item:
        root.child.count += 1
        return root.child
    elif root.child.sibling is None:
        node = create_node(root, item)
        root.child.sibling = node
        return root.child.sibling
    else:
        current_sibling = root.child.sibling
        last_sibling = None
        while current_sibling is not None:
            if current_sibling.label == item:
                current_sibling.count += 1
                return current_sibling
            else:
                last_sibling = current_sibling
                current_sibling = current_sibling.sibling
        node = create_node(root, item)
        last_sibling.sibling = node        
        return node

In [7]:
def pre_post(root, pre=0, post=0):
    root.pre_order = pre
    post_ = post
    if root.child is not None:
        root.child.pre_order = pre + 1
        pre, post_ = pre_post(root.child, root.child.pre_order, post)
    root.post_order = post_
    if root.sibling is not None:
        root.sibling.pre_order = pre + 1
        pre, post_ = pre_post(root.sibling, root.sibling.pre_order, root.post_order + 1)
        return pre, post_
    return pre, post_ + 1    

In [8]:
def build_ppc_tree(filename, itemset_1):
    root = PPCNode()
    with open(filename) as finput:
        f = finput.read().split('\n')
        for line in f:
            items = line.strip().split(' ')            
            transaction = []
            for item in items:
                if item in list(itemset_1.keys()):
                    transaction.append((item, itemset_1[item]))
                else:
                    print('removed:', item)
            transaction.sort(key=lambda x: x[1], reverse=True)
            root_aux = root
            for item in transaction:
                root_aux = insert_ppc_node(root_aux, item[0])
    pre_post(root)
    return root

In [9]:
def print_tree(root):
    print(root.label, root.pre_order, root.post_order)
    if root.child:
        print_tree(root.child)
    if root.sibling:
        print_tree(root.sibling)

In [10]:
def make_n_list(root, n_list={}):
    if root.label:
        if root.label not in n_list:
            n_list[root.label] = []
        n_list[root.label].append(((root.pre_order, root.post_order), root.count))
    if root.child:
        make_n_list(root.child, n_list)
    if root.sibling:
        make_n_list(root.sibling, n_list)

In [11]:
def NL_interserction(n_list1, n_list2, minsup):
    n_list_result = []
    for k in n_list1:
        for l in n_list2:
            if k[0][0] < l[0][0] and k[0][1] > l[0][1]:                 
                n_list_result.append((k[0], l[1]))
    d = {x:0 for x, _ in n_list_result} 
    for name, num in n_list_result: d[name] += num 
    n_list_result = list(map(tuple, d.items()))
    n_list_result = list(filter(lambda x: x[1] >= minsup, n_list_result))
    return n_list_result

In [12]:
def make_n2_list(n_list, items_ordered):
    global support
    global F
    n2_list = {}
    for i, x in enumerate(items_ordered):
        for j in range(i+1, len(items_ordered)):
            n_list_result = NL_interserction(n_list[x], n_list[items_ordered[j]], support)
            if len(n_list_result) > 0:
                key = '-'.join([x, items_ordered[j][0]])
                n2_list[key] = n_list_result
    return n2_list

In [13]:
def find_subsets(items, n):
    return list(itertools.combinations(items, n))


def get_all_subsets(items):
    subsets = [find_subsets(items, i) for i in range(len(items)+1)]
    subsets = ['-'.join(map(str, item)) for item in list(itertools.chain.from_iterable(subsets))]
    return subsets[1:]

In [14]:
def get_n_list(key):
    global all_n_list
    return all_n_list[key] if key in all_n_list else []

In [15]:
def building_pattern_tree(node, cad_items, parent_fit):
    global support
    global all_n_list
    global F
    global output_file
    
    node.equivalent_items = []
    node.child_nodes = []
    next_cad_items = []    
    p1 = node.itemset
    p1_n_list = get_n_list('-'.join(p1))
    for item in cad_items:
        p2 = [item] + p1[1:]
        p = [item] + p1
        p2_n_list = get_n_list('-'.join(p2))        
        p_n_list = NL_interserction(p2_n_list, p1_n_list, support)
        p_support = sum([item[1] for item in p_n_list])
        p_key = '-'.join(p)
        
        if len(p_n_list) > 0: # add n_list for new k-itemset            
            if p_key not in all_n_list:
                all_n_list[p_key] = []
            all_n_list[p_key] += p_n_list
            
        if p_support == node.support:
            node.equivalent_items += [item]
        elif p_support >= support:
            node_ = FPTNode()
            node_.label = item
            node_.itemset = p
            node_.support = p_support
            node.child_nodes += [node_]
            next_cad_items += [item]
    
    nd_fit = []        
    if len(node.equivalent_items) > 0:
        subsets = get_all_subsets(node.equivalent_items)
        cand_itemsets = [('-'.join([item, node.label]), node.support) for item in subsets]
        if len(parent_fit) == 0:
            nd_fit = cand_itemsets
        else:
            nd_fit = []
            for cand_item in cand_itemsets:
                for parent_item in parent_fit:
                    nd_fit.append(('-'.join([cand_item[0], parent_item[0]]), node.support))                    
        
        for x in nd_fit:
            output_file.write(x[0] + " #SUP: " + str(x[1]) + "\n")
                        

    if len(node.child_nodes) > 0:
        for n in node.child_nodes:
            aheads_ = [i for i in items_ordered[:items_ordered.index(n.label)] if i in next_cad_items]
            if aheads_ == []:
                output_file.write('-'.join(n.itemset) + " #SUP: " + str(n.support) + "\n")
            else:
                building_pattern_tree(n, aheads_, nd_fit)

In [16]:
def prepostplus():
    global all_n_list
    global output_file
    
    initial_time = time.time()
    itemset_1 = make_itemset_1(input_file)
    tree = build_ppc_tree(input_file, itemset_1)
    
    n_list = {}
    make_n_list(tree, n_list)
    
    F = list(itemset_1.items())
    F.sort(key=lambda x: x[1], reverse=True)
    
    for x in F:
        output_file.write(x[0] + " #SUP: " + str(x[1]) + "\n")
    items_ordered = [x[0] for x in F]
    
    n2_list = make_n2_list(n_list, items_ordered)
    
    all_n_list = copy.deepcopy(n2_list)
    
    for key, n_list in list(n2_list.items()):
        item = key[:key.index('-')]
        aheads = items_ordered[:items_ordered.index(item)]
        node = FPTNode()
        node.label = key
        node.itemset = key.split('-')
        node.support = sum([item[1] for item in n_list])
        output_file.write(key + " #SUP: " + str(node.support) + "\n")
        building_pattern_tree(node, aheads, [])
    final_time = time.time()
    
    #output_file.close()
    print("Execution time (s):", final_time - initial_time)

In [17]:
memory_usage((prepostplus))

removed: 4
Execution time (s): 0.0009775161743164062
removed: 4


ValueError: I/O operation on closed file.